# Baseline CNN for Snake Idenfitifcation


In [1]:
# imports
import torch
import torchvision
import random
import matplotlib.pyplot as plt
import os
import copy
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score
from PIL import Image

In [4]:
DATA_DIR = "../../teams/DSC180A_FA20_A00/a01group09/"
TRAIN_DIR = "../../teams/DSC180A_FA20_A00/a01group09/train_images/"
train_data = datasets.ImageFolder(TRAIN_DIR)
len(tran_data.classes)

RuntimeError: Found 0 files in subfolders of: ../../teams/DSC180A_FA20_A00/a01group09/train_images/
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif,.tiff,.webp